In [1]:
!pip install flask flask-ngrok pyngrok


In [3]:
pip install psycopg2-binary


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from flask import Flask, request, jsonify
import psycopg2
from datetime import datetime
import json

app = Flask(__name__)
TARGET_MINT = "9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump"

# Database credentials
DB_HOST = "localhost"
DB_NAME = "transactions"
DB_USER = "postgres"
DB_PASS = "25@Sathvik"
DB_PORT = 5432

def get_db_connection():
    return psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASS,
        port=DB_PORT
    )

def get_top_60_wallets():
    """Load top 60 wallets from DB. This should be cached or optimized in real use."""
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("""
            SELECT wallet FROM (
                SELECT wallet, SUM(amount) AS total_amount
                FROM transactions
                GROUP BY wallet
                ORDER BY total_amount DESC
                LIMIT 60
            ) AS top_wallets
        """)
        wallets = [row[0] for row in cursor.fetchall()]
        cursor.close()
        conn.close()
        return set(wallets)
    except Exception as e:
        print("❌ Error loading top wallets:", e)
        return set()

# def infer_protocol(txn_data):
#     events = txn_data.get("events", {})
#     instructions = events.get("instructions", [])
#     for inst in instructions:
#         if "jup" in str(inst).lower():
#             return "Jupiter"
#         if "raydium" in str(inst).lower():
#             return "Raydium"
#         if "orca" in str(inst).lower():
#             return "Orca"
#     return "Unknown"

def infer_protocol(txn_data):
    PROTOCOL_IDS = {
        "Jupiter": [
            "JUP4Fb2cqiRUcaTHdrPC8h2gNsA2ETXiPDD33WcGuJB",
            "JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4"
        ],
        "Raydium": [
            "675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8",
            "CPMMoo8L3F4NbTegBCKVNunggL7H1ZpdTHKxQB5qKP1C",
            "CAMMCzo5YL8w4VFF8KVHrK22GGUsp5vtaW7grrKgrWqK"
        ],
        "Orca": [
            "whirLbMiicVdio4qvUfM5KAg6Ct8VwpYzGff3uctyCc"
        ]
    }
    try:
        for inst in txn_data.get("instructions", []):
            pid = inst.get("programId", "")
            for proto, ids in PROTOCOL_IDS.items():
                if pid in ids:
                    return proto
        return "Unknown"
    except Exception as e:
        print("❌ Protocol detection error:", e)
        return "Unknown"

def insert_transaction(txn_dict):
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO transactions (timestamp, wallet, amount, direction, protocol, mint)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (
            txn_dict["timestamp"],
            txn_dict["wallet"],
            txn_dict["amount"],
            txn_dict["direction"],
            txn_dict["protocol"],
            txn_dict["mint"]
        ))
        conn.commit()
        cursor.close()
        conn.close()
        print("✅ Transaction inserted.")
    except Exception as e:
        print("❌ Insert error:", e)


@app.route('/webhook', methods=['POST'])
def webhook():
    data = request.json
    if not data:
        return jsonify({"error": "No data received"}), 400

    print("\n🎯 Webhook Event Received!")
    top_wallets = get_top_60_wallets()

    for txn in data:
        timestamp = datetime.utcnow()
        protocol = infer_protocol(txn)

        user_accounts = set()
        for account_info in txn.get("accountData", []):
            for token in account_info.get("tokenBalanceChanges", []):
                user = token.get("userAccount")
                if user:
                    user_accounts.add(user)

        for account_info in txn.get("accountData", []):
            for token in account_info.get("tokenBalanceChanges", []):
                mint = token.get("mint")
                if mint != TARGET_MINT:
                    continue

                user = token.get("userAccount")
                if user not in top_wallets:
                    continue  # skip non-top-wallet users

                raw_amount = int(token["rawTokenAmount"]["tokenAmount"])
                decimals = int(token["rawTokenAmount"]["decimals"])
                amount = raw_amount / (10 ** decimals)
                direction = "BUY" if raw_amount > 0 else "SELL"

                # Check internal vs external
                is_internal = all(participant in top_wallets for participant in user_accounts)
                if not is_internal and user not in top_wallets:
                    continue  # external, but we only care about the top wallet's side

                txn_dict = {
                    "timestamp": timestamp,
                    "wallet": user,
                    "amount": amount,
                    "direction": direction,
                    "protocol": protocol,
                    "mint": mint
                }

                print("📥 Inserting transaction:", txn_dict)
                insert_transaction(txn_dict)

    return jsonify({"status": "received"}), 200



if __name__ == '__main__':
    print("🚀 Starting Flask webhook on localhost with PostgreSQL...")
    app.run(host='0.0.0.0', port=8000)


🚀 Starting Flask webhook on localhost with PostgreSQL...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.165.77:8000
Press CTRL+C to quit



🎯 Webhook Event Received!
📥 Inserting transaction: {'timestamp': datetime.datetime(2025, 7, 9, 13, 44, 4, 181185), 'wallet': 'C9U2Ksk6KKWvLEeo5yUQ7Xu46X7NzeBJtd9PBfuXaUSM', 'amount': 569.022119, 'direction': 'BUY', 'protocol': 'Unknown', 'mint': '9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump'}
✅ Transaction inserted.
📥 Inserting transaction: {'timestamp': datetime.datetime(2025, 7, 9, 13, 44, 4, 181185), 'wallet': 'AnyiYYecsbzSW9P35R3dy6Js2KpPCxaF7vpspMeWS6bV', 'amount': -569.022119, 'direction': 'SELL', 'protocol': 'Unknown', 'mint': '9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump'}


C:\Users\Global\AppData\Local\Temp\ipykernel_5440\523517447.py:118: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow()
127.0.0.1 - - [09/Jul/2025 19:14:04] "POST /webhook HTTP/1.1" 200 -


✅ Transaction inserted.


...


🎯 Webhook Event Received!
📥 Inserting transaction: {'timestamp': datetime.datetime(2025, 7, 9, 17, 8, 57, 788062), 'wallet': 'C9U2Ksk6KKWvLEeo5yUQ7Xu46X7NzeBJtd9PBfuXaUSM', 'amount': -10.543517, 'direction': 'SELL', 'protocol': 'Unknown', 'mint': '9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump'}
✅ Transaction inserted.


127.0.0.1 - - [09/Jul/2025 22:38:58] "POST /webhook HTTP/1.1" 200 -



🎯 Webhook Event Received!


127.0.0.1 - - [09/Jul/2025 22:39:07] "POST /webhook HTTP/1.1" 200 -



🎯 Webhook Event Received!
